The code requires FEniCSx---see the README for details.

In [ ]:
%load_ext autoreload
%autoreload 2
# add path to code
import sys
sys.path.insert(0, '../source')

In [ ]:
import numpy as np
from main import solve
from ufl import SpatialCoordinate
from params import H, L, Nx, Nz, rho_w, g0, sigma0, u0
from dolfinx.fem import Expression, Function, FunctionSpace
from dolfinx.mesh import create_rectangle
from mpi4py import MPI
from fem_spaces import mixed_space
import matplotlib.pyplot as plt
from matplotlib import colors
from plotting import interp

Define domain:

In [ ]:
    # generate mesh
    p0 = [-L/2.0,-0.5*H]
    p1 = [L/2.0,0.5*H]
    domain = create_rectangle(MPI.COMM_WORLD,[p0,p1], [Nx, Nz])

Define melting rate function:

In [ ]:
m = lambda x,t: 0*x + 0*t

Define initial conditions

In [ ]:
P2 = FunctionSpace(domain, ("CG", 2))
P1 = FunctionSpace(domain, ("CG", 1))

In [ ]:
def step(x,val): 
    return val*(x[1] + abs(x[1]))/(2*abs(x[1])+1e-10)

In [ ]:
phii_0 = Function(P2)
phiw_0 = Function(P2)
u_0 = Function(P2)
w_0 = Function(P2)
pw_0 = Function(P1)
pe_0 = Function(P1)

In [ ]:
phii_0.interpolate(lambda x: step(x,1))
phiw_0.interpolate(lambda x: step(-x,0.25))
u_0.interpolate(lambda x: step(x,u0))
w_0.interpolate(lambda x: step(x,0))
pw_0.interpolate(lambda x: rho_w*g0*x[1] + sigma0)
pe_0.interpolate(lambda x: step(x,0)) 

In [ ]:
V = mixed_space(domain)
initial = Function(V)
initial.sub(0).sub(0).interpolate(u_0)
initial.sub(0).sub(1).interpolate(w_0)
initial.sub(1).interpolate(phii_0)
initial.sub(2).interpolate(phiw_0)
initial.sub(3).interpolate(pw_0)
initial.sub(4).interpolate(pe_0)

In [ ]:
phii = Function(P1)
phiw = Function(P1)

In [ ]:
phii.interpolate(Expression(initial.sub(1), P1.element.interpolation_points()))
phiw.interpolate(Expression(initial.sub(2), P1.element.interpolation_points()))

In [ ]:
# # PLOT initial volume fractions .... 
# X,Y,Phi_i = interp(phii,domain)
# X,Y,Phi_w = interp(phiw,domain)

# j=int(Nz/2.)
# plt.plot(Phi_i[:,j],Y[:,j],color='royalblue',linewidth=2,label=r'$\phi_\mathrm{i}$')
# plt.plot(Phi_w[:,j],Y[:,j],color='crimson',linewidth=2,label=r'$\phi_\mathrm{w}$')
# plt.legend(fontsize=16,loc='lower right')
# plt.ylim(-H/2.,H/2.)
# plt.xlabel(r'$\phi$',fontsize=18)
# plt.ylabel(r'$z$',fontsize=18)
# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
# plt.show()
# plt.close()

In [ ]:
###########################
# # contour plots ?!?!?
# plt.figure(figsize=(12,6))
# plt.subplot(121)
# plt.contourf(X,Y,Phi_i,cmap='Blues_r',levels=np.linspace(0,1,6),alpha=0.75)
# plt.xlabel(r'$x$',fontsize=18)
# plt.ylabel(r'$z$',fontsize=18)
# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
# cbar = plt.colorbar(orientation='horizontal')
# cbar.set_label(r'$\phi_\mathrm{i}$',fontsize=18)

# norm2 =colors.TwoSlopeNorm(vmin=0, vcenter=0.9, vmax=1)
# plt.subplot(122)
# plt.contourf(X,Y,Phi_w,cmap='Blues',levels=np.linspace(0,1,6),alpha=0.75)
# plt.xlabel(r'$x$',fontsize=18)
# plt.ylabel(r'$z$',fontsize=18)
# plt.gca().yaxis.tick_right()
# plt.gca().yaxis.set_label_position("right")
# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
# cbar = plt.colorbar(orientation='horizontal')
# cbar.set_label(r'$\phi_\mathrm{w}$',fontsize=18)

# plt.show()
# plt.close()